### Import useful libraries and helpers

In [2]:
%run "Imports.ipynb"
%run "Helpers.ipynb"

### Load and prepare the data

In [3]:
df_all = pd.read_pickle("df_all.pkl")

### Divide into train and test sets

In [30]:
# Skip rows with no prediction available
df_only_pred = df_all[['supply','forecast','zone','time', 'weekday','sat_or_sun','holidays']]
df_only_pred = df_only_pred.dropna()
df_only_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 934958 entries, 8986 to 948014
Data columns (total 7 columns):
supply        934958 non-null int64
forecast      934958 non-null float64
zone          934958 non-null object
time          934958 non-null object
weekday       934958 non-null object
sat_or_sun    934958 non-null int64
holidays      934958 non-null int64
dtypes: float64(1), int64(3), object(3)
memory usage: 57.1+ MB


In [31]:
# Create categorical variables
cat_feats = ['zone', 'time', 'weekday']
for feat in cat_feats:
    feat_id = feat + '_id'
    df_only_pred[feat_id] = df_only_pred[feat].factorize()[0]

In [32]:
# Divide into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_only_pred[['zone_id','time_id','weekday_id','forecast','sat_or_sun', 'holidays']], df_only_pred['supply'], test_size=0.2, random_state=123)

In [33]:
X_train_with_forecast = X_train
X_test_with_forecast = X_test

In [34]:
X_train = X_train[['zone_id', 'time_id', 'weekday_id','sat_or_sun', 'holidays']]
X_test = X_test[['zone_id', 'time_id', 'weekday_id','sat_or_sun', 'holidays']]

### Naive prediction

In [35]:
success_metric(y_test,X_test_with_forecast['forecast'])

0.6370893498

In [36]:
r2_score(y_test,X_test_with_forecast['forecast'])

0.7511082945852964

### XGBRegressor

In [37]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [38]:
xg_reg.fit(X_train,y_train)

forecast = xg_reg.predict(X_test)

/Users/asia/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



In [39]:
success_metric(y_test,forecast)

0.9553096568

In [40]:
r2_score(y_test,forecast)

0.31045063243803284